<a href="https://colab.research.google.com/github/kenyuisme/model-evaluation-notebook/blob/main/Carta_Healthcare_Demo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install dataframe-image
!pip install fpdf

In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, precision_recall_curve, average_precision_score, cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
from PIL import Image
import dataframe_image as dfi


In [33]:
def plot_pr_curves_all_models(gt, model_outputs, attribute, filename='pr_curve', append=False):
    plt.figure(figsize=(8,6))

    for model_name, probs in model_outputs.items():
        y_true = gt[attribute]
        y_scores = probs[attribute]

        precision, recall, _ = precision_recall_curve(y_true, y_scores)
        ap = average_precision_score(y_true, y_scores)

        plt.plot(recall, precision, label=f'{model_name} (AP={ap:.2f})')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curves for "{attribute}"')
    plt.legend(loc='lower left')
    plt.grid(True)
    plt.savefig(f'{filename}.png')
    if append:
      image_files.append(f'{filename}.png')

def plot_heatmap(summary_df, metric="F1", figsize=(10, 6), cmap="Spectral", append=False):
    """
    Plot a heatmap of a chosen metric across models and attributes with color scale fixed between 0 and 1.

    Parameters:
    - summary_df: DataFrame with columns ['Model', 'Attribute', 'Precision', 'Recall', 'F1', 'Accuracy']
    - metric: Which metric to visualize ("Precision", "Recall", "F1", "Accuracy")
    - figsize: Tuple for figure size
    - cmap: Colormap for heatmap (default "Spectral" for colorblind-friendly)
    """
    if metric not in summary.columns:
        raise ValueError(f"Metric '{metric}' is not valid. Choose from {summary.columns}")

    pivot_df = summary_df.pivot(index="Attribute", columns="Model", values=metric)

    plt.figure(figsize=figsize)
    sns.heatmap(
        pivot_df,
        annot=True,
        fmt=".2f",
        cmap=cmap,
        cbar_kws={'label': metric},
        linewidths=0.5,
        linecolor='gray',
        vmin=0,    # force color scale min
        vmax=1     # force color scale max
    )
    plt.title(f"{metric} Heatmap by Attribute and Model", fontsize=14)
    plt.xlabel("Model")
    plt.ylabel("Attribute")
    plt.tight_layout()
    plt.savefig(f'{metric} heatmap.png')
    if append:
      image_files.append(f'{metric} heatmap.png')
    plt.show()
    plt.close()


def plot_horizontal_bar(summary_df, metric="F1", figsize=(10, 6), filename='bar_chart', append=False):
    """
    Plot a horizontal grouped bar chart of a chosen metric across attributes and models.

    Parameters:
    - summary_df: DataFrame with columns ['Model', 'Attribute', 'Precision', 'Recall', 'F1', 'Accuracy']
    - metric: Which metric to visualize ("Precision", "Recall", "F1", "Accuracy")
    - figsize: Tuple for figure size
    """
    if metric not in summary.columns:
        raise ValueError(f"Metric '{metric}' is not valid. Choose from {summary.columns}.")

    # Pivot the data so rows = attributes, columns = models, values = metric
    pivot_df = summary_df.pivot(index="Attribute", columns="Model", values=metric)

    # Sort attributes by average metric value for better readability
    pivot_df['mean_metric'] = pivot_df.mean(axis=1)
    pivot_df = pivot_df.sort_values('mean_metric', ascending=True)
    pivot_df = pivot_df.drop(columns=['mean_metric'])

    attributes = pivot_df.index.tolist()
    models = pivot_df.columns.tolist()

    y_pos = np.arange(len(attributes))
    bar_height = 0.8 / len(models)  # bar thickness adjusted for number of models

    fig, ax = plt.subplots(figsize=figsize)

    for i, model in enumerate(models):
        # Calculate position for each model's bar within each attribute row
        positions = y_pos - 0.4 + i * bar_height + bar_height/2
        ax.barh(positions, pivot_df[model], height=bar_height, label=model)

    ax.set_yticks(y_pos)
    ax.set_yticklabels(attributes)
    ax.set_xlim(0, 1)  # force x-axis between 0 and 1 for metric scale
    ax.set_xlabel(metric)
    ax.set_title(f"{metric} by Attribute and Model")
    ax.legend(title="Model")
    plt.tight_layout()
    plt.savefig(f'{metric} {filename}.png')
    if append:
      image_files.append(f'{metric} {filename}.png')

def summarize_metrics(gt, model_outputs, thresholds, coverage_threshold=0.8):
    """
    Summarize precision, recall, F1, accuracy, average precision, Cohen's Kappa,
    and print model coverage.

    Parameters:
        gt (DataFrame): Ground truth DataFrame with 'case_id' and attributes as columns.
        model_outputs (dict): Dict of model names to DataFrames of predicted probabilities.
        thresholds (dict): Dict mapping attribute names to thresholds.
        coverage_threshold (float): Minimum value for all metrics to consider an attribute covered.

    Returns:
        DataFrame: Summary metrics.
    """
    from collections import defaultdict

    attributes = [col for col in gt.columns if col != 'case_id']
    rows = []
    coverage_tracker = defaultdict(lambda: {'Covered': 0, 'Total': 0})

    for model_name, probs in model_outputs.items():
        for attr in attributes:
            y_true = gt[attr]
            y_scores = probs[attr]
            threshold = thresholds.get(attr, 0.5)
            y_pred = (y_scores >= threshold).astype(int)

            precision = precision_score(y_true, y_pred, zero_division=0)
            recall = recall_score(y_true, y_pred, zero_division=0)
            f1 = f1_score(y_true, y_pred, zero_division=0)
            accuracy = accuracy_score(y_true, y_pred)
            avg_prec = average_precision_score(y_true, y_scores)
            kappa = cohen_kappa_score(y_true, y_pred)

            # Determine if attribute is "covered"
            metrics = [precision, recall, f1, accuracy, avg_prec, kappa]
            is_covered = all(m >= coverage_threshold for m in metrics)
            coverage_status = "Covered" if is_covered else "Not Covered"

            coverage_tracker[model_name]['Total'] += 1
            if is_covered:
                coverage_tracker[model_name]['Covered'] += 1

            rows.append({
                'Model': model_name,
                'Attribute': attr,
                'Threshold': threshold,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Accuracy': accuracy,
                'Average Precision (AP)': avg_prec,
                'Cohen\'s Kappa': kappa,
                'Coverage Status': coverage_status
            })

    # Print coverage summary for each model
    print("\n=== Model Coverage Summary ===")
    for model_name, counts in coverage_tracker.items():
        coverage_pct = counts['Covered'] / counts['Total'] * 100
        print(f"{model_name}: {counts['Covered']} of {counts['Total']} attributes covered ({coverage_pct:.1f}%)")
    print("==============================\n")

    summary_df = pd.DataFrame(rows)
    return summary_df




def styled_metrics_by_model(summary_df, filename='summary_metrics.png', append=False, show=True):
    import numpy as np

    def color_rows_by_model(row):
        # Assign a color based on model name hash (consistent but different per model)
        colors = ['#E3F2FD', '#FFF3E0', '#E8F5E9', '#F3E5F5', '#FFEBEE']
        model_list = summary_df['Model'].unique()
        color_map = {model: colors[i % len(colors)] for i, model in enumerate(model_list)}
        return [f'background-color: {color_map[row["Model"]]}'] * len(row)

    def traffic_light(val):
        """Apply traffic light colors based on value thresholds"""
        if pd.isna(val):
            return ''
        elif val >= 0.8:
            return 'background-color: #C8E6C9'  # Green
        elif val >= 0.6:
            return 'background-color: #FFF9C4'  # Yellow
        else:
            return 'background-color: #FFCDD2'  # Red

    # Metrics to apply traffic light coloring to
    metrics_cols = ['Precision', 'Recall', 'F1', 'Accuracy', 'Average Precision (AP)', 'Cohen\'s Kappa']

    # Calculate coverage per model
    print("\n=== Model Coverage Summary ===")
    for model_name in summary_df['Model'].unique():
        model_df = summary_df[summary_df['Model'] == model_name]
        # An attribute is "covered" if all metrics >= 0.8
        covered_attrs = model_df[metrics_cols].ge(0.8).all(axis=1).sum()
        total_attrs = len(model_df)
        coverage_pct = (covered_attrs / total_attrs) * 100
        print(f"{model_name}: {covered_attrs} of {total_attrs} attributes covered ({coverage_pct:.1f}%)")
    print("==============================\n")

    # Style the DataFrame
    styled = summary_df.style \
        .apply(color_rows_by_model, axis=1) \
        .map(traffic_light, subset=metrics_cols) \
        .format(precision=2) \
        .set_properties(**{'text-align': 'center'}) \
        .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

    if show:
        display(styled)

    if filename:
        dfi.export(styled, filename, table_conversion="matplotlib")

    if append:
        image_files.append(filename)



def create_pdf_from_images(image_files, output_pdf="model_evaluation_report.pdf"):
    pdf = FPDF()

    for img_file in image_files:
        cover = Image.open(img_file)
        width, height = cover.size

        # Convert pixels to mm with 1px=0.264583 mm
        width, height = width * 0.264583, height * 0.264583

        # Ensure page is portrait
        orientation = 'P' if width <= height else 'L'
        pdf.add_page(orientation=orientation)

        # Resize image to fit page
        pdf.image(img_file, x=0, y=0, w=210 if orientation=='P' else 297)

    pdf.output(output_pdf, "F")
    print(f"PDF report saved as {output_pdf}")

In [23]:
# Sample case IDs
case_ids = [f'case_{i+1}' for i in range(1000)]

# Attributes
attributes = ['comorbidity_diabetes', 'comorbidity_hypertension', 'medication_aspirin', 'mortality_30_day']

# Generate ground truth (binary)
np.random.seed(42)
ground_truth_data = {'case_id': case_ids}
for attr in attributes:
    ground_truth_data[attr] = np.random.choice([0,1], size=1000, p=[0.7,0.3])
ground_truth_df = pd.DataFrame(ground_truth_data)
ground_truth_df.to_csv('ground_truth.csv', index=False)

# Generate probabilistic model outputs with noise
def generate_model_output_probs(gt_df, noise_level=0.1, model_name='model'):
    np.random.seed(hash(model_name) % 2**32)
    model_data = {'case_id': gt_df['case_id']}
    for attr in attributes:
        probs = gt_df[attr] * (1 - noise_level) + (1 - gt_df[attr]) * noise_level
        probs = probs + np.random.normal(0, noise_level/2, size=len(probs))
        probs = np.clip(probs, 0, 1)
        model_data[attr] = probs
    return pd.DataFrame(model_data)

model_1_df = generate_model_output_probs(ground_truth_df, noise_level=0.35, model_name='Model_1')
model_2_df = generate_model_output_probs(ground_truth_df, noise_level=0.37, model_name='Model_2')
model_3_df = generate_model_output_probs(ground_truth_df, noise_level=0.39, model_name='Model_3')

model_1_df.to_csv('model_output_1.csv', index=False)
model_2_df.to_csv('model_output_2.csv', index=False)
model_3_df.to_csv('model_output_3.csv', index=False)

#Load Data

In [24]:
# Load data
gt_df = pd.read_csv('ground_truth.csv')
model_outputs = {
    'Model_1': pd.read_csv('model_output_1.csv'),
    'Model_2': pd.read_csv('model_output_2.csv'),
    'Model_3': pd.read_csv('model_output_3.csv'),
}

attributes = ['comorbidity_diabetes', 'comorbidity_hypertension', 'medication_aspirin', 'mortality_30_day']



In [25]:
threshold = {
    'comorbidity_diabetes': 0.7,
    'comorbidity_hypertension': 0.5,
    'medication_aspirin': 0.5,
    'mortality_30_day': 0.5
}

In [26]:
image_files = []

In [29]:
summary = summarize_metrics(ground_truth_df, model_outputs, threshold)
print(summary)


=== Model Coverage Summary ===
Model_1: 0 of 4 attributes covered (0.0%)
Model_2: 0 of 4 attributes covered (0.0%)
Model_3: 0 of 4 attributes covered (0.0%)

      Model                 Attribute  Threshold  Precision    Recall  \
0   Model_1      comorbidity_diabetes        0.7   0.885496  0.402778   
1   Model_1  comorbidity_hypertension        0.5   0.673797  0.820847   
2   Model_1        medication_aspirin        0.5   0.650259  0.807074   
3   Model_1          mortality_30_day        0.5   0.606145  0.772242   
4   Model_2      comorbidity_diabetes        0.7   0.786885  0.333333   
5   Model_2  comorbidity_hypertension        0.5   0.621134  0.785016   
6   Model_2        medication_aspirin        0.5   0.619165  0.810289   
7   Model_2          mortality_30_day        0.5   0.512064  0.679715   
8   Model_3      comorbidity_diabetes        0.7   0.746377  0.357639   
9   Model_3  comorbidity_hypertension        0.5   0.527578  0.716612   
10  Model_3        medication_aspirin 

In [34]:
styled_metrics_by_model(summary, append=True)


=== Model Coverage Summary ===
Model_1: 0 of 4 attributes covered (0.0%)
Model_2: 0 of 4 attributes covered (0.0%)
Model_3: 0 of 4 attributes covered (0.0%)



,Model,Attribute,Threshold,Precision,Recall,F1,Accuracy,Average Precision (AP),Cohen's Kappa,Coverage Status
0,Model_1,comorbidity_diabetes,0.70,0.89,0.40,0.55,0.81,0.81,0.46,Not Covered
1,Model_1,comorbidity_hypertension,0.50,0.67,0.82,0.74,0.82,0.84,0.61,Not Covered
2,Model_1,medication_aspirin,0.50,0.65,0.81,0.72,0.81,0.83,0.57,Not Covered
3,Model_1,mortality_30_day,0.50,0.61,0.77,0.68,0.80,0.73,0.53,Not Covered
4,Model_2,comorbidity_diabetes,0.70,0.79,0.33,0.47,0.78,0.67,0.36,Not Covered
5,Model_2,comorbidity_hypertension,0.50,0.62,0.79,0.69,0.79,0.73,0.53,Not Covered
6,Model_2,medication_aspirin,0.50,0.62,0.81,0.70,0.79,0.72,0.54,Not Covered
7,Model_2,mortality_30_day,0.50,0.51,0.68,0.58,0.73,0.65,0.39,Not Covered
8,Model_3,comorbidity_diabetes,0.70,0.75,0.36,0.48,0.78,0.63,0.37,Not Covered
9,Model_3,comorbidity_hypertension,0.50,0.53,0.72,0.61,0.72,0.66,0.39,Not Covered


In [ ]:
plot_heatmap(summary, metric="Cohen\'s Kappa", figsize=(10, 6), cmap="Spectral")
plt.show()
plt.close()

In [ ]:
plot_horizontal_bar(summary, metric="F1", figsize=(10, 6), append=True)
plt.show()
plt.close()

In [ ]:
for attribute in attributes:
  plot_pr_curves_all_models(gt_df, model_outputs, attribute, filename=f'{attribute} pr_curve.png', append=True)
  plt.show()
  plt.close()

In [ ]:
def generate_image_files(visuals={
    'pr_curve': True,
    'bar_chart': True,
    'summary_table': True
}):
  image_files = []
  if visuals['pr_curve']:
    for attribute in attributes:
      plot_pr_curves_all_models(gt_df, model_outputs, attribute, filename=f'{attribute} pr_curve', append=True)
      plt.close()
  if visuals['bar_chart']:
    for metric in metrics:
      plot_horizontal_bar(summary, metric=metric, figsize=(10, 6), filename='bar_chart', append=True)
      plt.close()
  if visuals['summary_table']:
    styled_metrics_by_model(summary, append=True, show=False)


In [ ]:
attributes = ['comorbidity_diabetes', 'comorbidity_hypertension', 'medication_aspirin', 'mortality_30_day']
metrics = ['Precision', 'Recall', 'F1', 'Accuracy', 'Average Precision (AP)', 'Cohen\'s Kappa']
visuals = {
    'pr_curve': True,
    'bar_chart': True,
    'summary_table': True
}

In [ ]:
image_files = []
generate_image_files(visuals=visuals)

In [ ]:
image_files

In [ ]:
create_pdf_from_images(image_files)